In [2]:
import gym
import numpy as np
import random
import torch
import time
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
map = ['FF',
        'SG']


In [4]:
env = gym.make('FrozenLake-v0', is_slippery=False, desc = map)


# visualize 4x4 frozen lake
env.render()


FF
SG


In [5]:
# Total number of States and Actions
n_states = env.observation_space.n
n_actions = env.action_space.n
n_rows = 2
n_cols = 2
print( "States = ", n_states)
print( "Actions = ", n_actions)

States =  4
Actions =  4


In [7]:
def choose_action(Q, state, epsilon):
  random_for_epsilon = np.random.rand()
  if random_for_epsilon <= epsilon:
    s = Q.loc[state].notna()
    vals = s[s].index.values
    action = random.choice(vals)
  else: 
    Q.loc[state] += np.random.rand(n_actions,)/100
    action = np.argmax(Q.loc[state])
  return action

In [8]:
##assign index to each state using state-matrix

state_matrix = np.arange(0,n_states).reshape(n_rows,n_cols)
state_matrix

def rowsandcols(state):
  ''' input: state returned by env
      output: location of state as (row,col) tuple'''
  return int(np.where(state_matrix ==state)[0]), int(np.where(state_matrix ==state)[1])

In [34]:
reps = 100
num_episodes = 100

In [38]:
#n_successes = []

steps_all_reps= []

for i_rep in range(reps):
  

  Q = pd.DataFrame([[np.NaN, np.random.rand(), np.random.rand(), np.NaN ],
             [np.random.rand(),  np.random.rand(), np.NaN, np.NaN],
              [np.NaN, np.NaN, -0.7, np.random.rand()],
              [np.random.rand()*0.001, np.random.rand()*0.001, np.random.rand()*0.001, np.random.rand()*0.001]])


  steps_total = [] # store number of steps taken in each episode
  rewards_total = [] #store reward obtained for each episode
  epsilon_total = [] #store epsilon obtained at the end of each episode
  terminal_state = [] 


  epsilon = 0.2
  epsilon_final = 0.1
  epsilon_decay = 0.99
  gamma = 0.90 # discount factor
  learning_rate = 0.9 #how important is the difference between q-val from q-table and what's observed

  for i_episode in range(num_episodes):
    
    # resets the environment
    state = env.reset()
    step = 0
    reward = 0

  ## as epsilon decays with more timesteps, the prob. of selecting a random val < e decays --> more likely to exploit. 
    if epsilon > epsilon_final:
            epsilon *= epsilon_decay

    while True:
        
        step += 1
        
        action = choose_action(Q, state, epsilon)
  
         
        ## env gives reward and next state and whether we've reached terminal state upon taking action at current state.. 
        new_state, reward , done, info = env.step(action)

        ##if you want reward penalized at for each timestep
        #reward = rewarder(new_state, reward)

        # filling the Q Table - 
        
        Q.loc[state, action] = (1- learning_rate)*Q.at[state, action] + learning_rate*(reward + gamma * np.max(Q.loc[new_state]))
        
        # Setting new state for next action
        state = new_state
        tile = map[rowsandcols(state)[0]][rowsandcols(state)[1]]
        #env.render()
        
        if done:
          #print(Q)
          
          terminal_state.append(tile)
          steps_total.append(step)
          rewards_total.append(reward)
          epsilon_total.append(epsilon)
          #print('Episode: {} Reward: {} Steps Taken: {} Terminal State: {}, Epsilon: {}'.format(i_episode,reward, step, tile, epsilon))
          break
  steps_all_reps.append(steps_total)
  #n_successes.append(terminal_state.count('G'))

In [42]:
mean_rep_step = np.mean(steps_all_reps, axis = 0)

In [44]:
np.std(steps_all_reps, axis = 0)

array([9.18718673, 2.05873748, 1.47986486, 1.33626345, 1.30766968,
       1.47986486, 1.07777549, 1.08277421, 1.50983443, 1.28934092,
       1.39985714, 1.14455231, 1.38260623, 1.36747943, 1.03923048,
       1.14175304, 1.        , 1.28436755, 0.86625631, 0.80099938,
       0.80274529, 0.71414284, 0.88181631, 0.64992307, 0.7825599 ,
       0.84947042, 0.57236352, 0.68673139, 0.6       , 0.72111026,
       0.61188234, 0.64992307, 0.47497368, 0.92606695, 0.70738957,
       0.9335952 , 0.63843559, 0.72111026, 0.7825599 , 0.58343809,
       0.6       , 0.82462113, 0.5425864 , 0.51029403, 0.58343809,
       0.51961524, 0.552811  , 0.63843559, 0.5425864 , 0.58343809,
       0.34117444, 0.34117444, 0.34117444, 0.61188234, 0.51029403,
       0.66332496, 0.51029403, 0.34117444, 0.39191836, 0.47497368,
       0.34117444, 0.69828361, 0.57236352, 0.51029403, 0.63843559,
       0.69828361, 0.39191836, 0.47497368, 0.51029403, 0.8138796 ,
       0.552811  , 0.58343809, 0.69828361, 0.58343809, 0.43588

In [46]:
import plotly.express as px

In [56]:
fig = px.bar( x= np.arange(1,101), y= mean_rep_step, width=800, height=400)
fig.update_layout(
    title=" Average steps taken in an episode out of 100 reps ",
    xaxis_title='episode',
    #x_axis_range = [0,101],
    yaxis_title=" average step taken in an episode",
    yaxis_range=[0,11],
    font=dict(
        family="Courier New, monospace",
        size=15))


fig.show()

In [57]:
Q

,0,1,2,3
0,NaN,0.901572,0.741194,NaN
1,0.812408,0.264836,NaN,NaN
2,NaN,NaN,1.001771,0.870048
3,0.000741,0.000424,0.000702,0.000880
